### 变形

In [1]:
import numpy as np
import pandas as pd


In [2]:
pd.DataFrame({"gender":["F","F","M","M","F"],
             "height":[165,163,180,178,150]})


,gender,height
0,F,165
1,F,163
2,M,180
3,M,178
4,F,150


In [4]:
pd.DataFrame({"height:F":[163,160,],
             "height:M":[180,175]})

,height:F,height:M
0,163,180
1,160,175


### pivot 是一种典型的长表变宽表的函数

In [5]:
df = pd.DataFrame({'Class':[1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})

In [6]:
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [7]:
df.pivot(index='Name', columns='Subject', values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


In [8]:
df.loc[1, 'Subject'] = 'Chinese'

In [9]:
try:
    df.pivot(index='Name', columns='Subject', values='Grade')
except Exception as e:
    Err_Msg = e

In [10]:
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

In [11]:
df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                           'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                   'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                              'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})

In [12]:
df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [13]:
pivot_multi = df.pivot(index = ['Class', 'Name'],
                       columns = ['Subject','Examination'],
                       values = ['Grade','rank'])

In [14]:
pivot_multi

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

### pivot_table
pivot 的使用依赖于唯一性条件，那如果不满足唯一性条件，那么必须通过聚合操作使得相同行列组合对应的多个值变为一个值

In [15]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang',
                           'San Zhang', 'San Zhang',
                           'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                              'Chinese', 'Chinese', 'Math', 'Math'],
                   'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})

In [16]:
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


#### pandas 中提供了 pivot_table 来实现，其中的 aggfunc 参数就是使用的聚合函数

In [17]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


#### aggfunc可以用匿名函数来写

In [19]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = lambda x:x.mean())

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


#### pivot_table 具有边际汇总的功能，可以通过设置 margins=True 来实现，其中边际的聚合方式与 aggfunc 中给出的聚合方法一致

In [20]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc='mean',
               margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85,95.0,90.00
Si Li,75,90.0,82.50
All,80,92.5,86.25


### melt

In [21]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})

In [22]:
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [23]:
df_melted = df.melt(id_vars = ['Class', 'Name'],
                    value_vars = ['Chinese', 'Math'],
                    var_name = 'Subject',
                    value_name = 'Grade')

In [25]:
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


### 也可以通过pivot操作把df_melted转回df形式

In [28]:
df_unmelted = df_melted.pivot(index = ['Class', 'Name'],
                              columns='Subject',
                              values='Grade')

In [29]:
df_unmelted

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,80
2,Si Li,90,75


In [30]:
df_unmelted = df_unmelted.reset_index().rename_axis(columns={'Subject':''})

In [31]:
df_unmelted.equals(df)

True

 ### wide_to_long
 #### melt 方法中，在列索引中被压缩的一组值对应的列元素只能代表同一层次的含义，即 values_name 。现在如果列中包含了交叉类别，比如期中期末的类别和语文数学的类别，那么想要把 values_name 对应的 Grade 扩充为两列分别对应语文分数和数学分数，只把期中期末的信息压缩，这种需求下就要使用 wide_to_long 函数来完成

In [32]:
df = pd.DataFrame({'Class':[1,2],'Name':['San Zhang', 'Si Li'],
                   'Chinese_Mid':[80, 75], 'Math_Mid':[90, 85],
                   'Chinese_Final':[80, 75], 'Math_Final':[90, 85]})

In [33]:
df

,Class,Name,Chinese_Mid,Math_Mid,Chinese_Final,Math_Final
0,1,San Zhang,80,90,80,90
1,2,Si Li,75,85,75,85


In [34]:
pd.wide_to_long(df,
                stubnames=['Chinese', 'Math'],
                i = ['Class', 'Name'],
                j='Examination',
                sep='_',
                suffix='.+')

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

In [35]:
res = pivot_multi.copy()

In [36]:
res.columns = res.columns.map(lambda x:'_'.join(x))

In [37]:
res = res.reset_index()

In [38]:
res = pd.wide_to_long(res, stubnames=['Grade', 'rank'],
                      i = ['Class', 'Name'],
                      j = 'Subject_Examination',
                      sep = '_',
                      suffix = '.+')

In [39]:
res = res.reset_index()

In [40]:
res[['Subject', 'Examination']] = res[
    'Subject_Examination'].str.split('_', expand=True)

In [41]:
res = res[['Class', 'Name', 'Examination',
           'Subject', 'Grade', 'rank']].sort_values('Subject')

In [42]:
res = res.reset_index(drop=True)

In [43]:
res

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


### 索引的变形
#### stack与unstack

In [44]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['col_1', 'col_2'])

In [45]:
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [46]:
df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [47]:
df.unstack(2)

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [48]:
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

类似于 pivot 中的唯一性要求，在 unstack 中必须保证 被转为列索引的行索引层 和 被保留的行索引层 构成的组合是唯一的，例如把前两个列索引改成相同的破坏唯一性，那么就会报错：

In [49]:
my_index = df.index.to_list()

In [50]:
my_index[1] = my_index[0]

In [51]:
df.index = pd.Index(my_index)

In [52]:
df

col_1  col_2
A cat big      1.0    1.0
      big      1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [53]:
try:
    df.unstack()
except Exception as e:
    Err_Msg = e

In [54]:
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

In [55]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A', 'cat', 'big'),
                                    ('A', 'dog', 'small'),
                                    ('B', 'cat', 'big'),
                                    ('B', 'dog', 'small')]),
                  columns=['index_1', 'index_2']).T

In [56]:
df

A          B      
         cat   dog  cat   dog
         big small  big small
index_1  1.0   1.0  1.0   1.0
index_2  1.0   1.0  1.0   1.0

In [57]:
df.stack()

A         B     
               cat  dog  cat  dog
index_1 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0
index_2 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0

In [58]:
df.stack([1, 2])

A    B
index_1 cat big    1.0  1.0
        dog small  1.0  1.0
index_2 cat big    1.0  1.0
        dog small  1.0  1.0

#### 聚合与变形的关系
在上面介绍的所有函数中，除了带有聚合效果的 pivot_table 以外，所有的函数在变形前后并不会带来 values 个数的改变，只是这些值在呈现的形式上发生了变化。在上一章讨论的分组聚合操作，由于生成了新的行列索引，因此必然也属于某种特殊的变形操作，但由于聚合之后把原来的多个值变为了一个值，因此 values 的个数产生了变化，这也是分组聚合与变形函数的最大区别。

### 其他变形函数
#### crosstab，并不是一个值得推荐使用的函数，因为它能实现的所有功能 pivot_table 都能完成，并且速度更快。在默认状态下， crosstab 可以统计元素组合出现的频数，即 count 操作

In [62]:
df = pd.read_csv('train_k.csv')
pd.crosstab(index = df.Churn, columns = df.Transfer)

AttributeError: 'DataFrame' object has no attribute 'Transfer'

In [64]:
 pd.crosstab(index = df.Churn, columns = df.Transfer,
             values = [0]*df.shape[0], aggfunc = 'count')

AttributeError: 'DataFrame' object has no attribute 'Transfer'

In [65]:
df.pivot_table(index = '',
               columns = 'Transfer',
               values = 'Name',
               aggfunc = 'count')

KeyError: 'Name'

###  explode
explode 参数能够对某一列的元素进行纵向的展开，被展开的单元格必须存储 list, tuple, Series, np.ndarray 中的一种类型

In [66]:
df_ex = pd.DataFrame({'A': [[1, 2],'my_str',{1, 2},pd.Series([3, 4])],'B': 1})

In [67]:
df_ex.explode('A')

,A,B
0,1,1
0,2,1
1,my_str,1
2,"{1, 2}",1
3,3,1
3,4,1


### get_dummies
get_dummies 是用于特征构建的重要函数之一，其作用是把类别特征转为指示变量

In [69]:
pd.get_dummies(df.Churn).head()

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,0,1
